In [6]:
import json
import pandas as pd
import numpy as np
import pickle

In [3]:
with open('spider_data/train_spider.json') as f:
    query_data = json.load(f)

with open('spider_data/tables.json') as f:
    table_data = json.load(f)

multi_table_queries = [query for query in query_data if len(query['sql']['from']['table_units'])>1]

In [12]:
with open('spider_gold_mt.sql','w') as f:
    for rec in multi_table_queries[:500]:
        q = rec['query'].replace('\t',' ')
        line = f"{q}\t{rec['db_id']}\n"
        f.write(line)

In [8]:
with open('results_mt.pkl','rb') as f:
    results_mt = pickle.load(f)

In [11]:
pred_queries = []
for query in results_mt[:500]:
    if query.lower().find('select')!=-1:
        filtered_query = query[query.lower().find('select'):query.find(';')]
        pred_queries.append(filtered_query)
    else:
        pred_queries.append(-1)

with open('pred_mt.sql','w') as f:
    for rec in pred_queries:
        line = f"{rec}\n"
        f.write(line)        

In [17]:
pred_queries = []
for query in results_mt[:500]:
    if query.lower().find('select')!=-1:
        end_index1 = query.find(';') if query.find(';')>0  else len(query)
        end_index2 = query.find('\n') if query.find('\n')>0 else len(query)
        
        filtered_query = query[query.lower().find('select'):min(end_index1,end_index2)]
        pred_queries.append(filtered_query)
    else:
        pred_queries.append(-1)

with open('pred_mt.sql','w') as f:
    for rec in pred_queries:
        line = f"{rec}\n"
        f.write(line)

In [14]:
import pickle
with open('results_preprocessing_mt.pkl', 'rb') as f:
    res_codegen = pickle.load(f)

In [15]:
pred_codegen_queries = []
for query in res_codegen:
    if query.lower().find('select')!=-1:
        end_index1 = query.find(';') if query.find(';')>0  else len(query)
        end_index2 = query.find('\n') if query.find('\n')>0 else len(query)
        
        filtered_query = query[query.lower().find('select'):min(end_index1,end_index2)]
        pred_codegen_queries.append(filtered_query)
    else:
        pred_codegen_queries.append(-1)

with open('pred_codegen_mt.sql','w') as f:
    for rec in pred_codegen_queries:
        line = f"{rec}\n"
        f.write(line)

## unaliasing

In [29]:
import sqlglot
ua_queries = []
fails = 0
fails_list = []

for query in pred_codegen_queries:
    try:
        qt = sqlglot.parse_one(query)
        for t in qt.find_all(sqlglot.exp.Alias):
            t.replace(t.unalias())
        ua_queries.append(qt.sql())
    except:
        fails+=1
        fails_list.append(query)
        ua_queries.append(query)

Argument 'format' is not supported for expression 'ToChar' when targeting Dialect.
Argument 'format' is not supported for expression 'ToChar' when targeting Dialect.
Argument 'format' is not supported for expression 'ToChar' when targeting Dialect.
Argument 'format' is not supported for expression 'ToChar' when targeting Dialect.
Argument 'format' is not supported for expression 'ToChar' when targeting Dialect.
Argument 'format' is not supported for expression 'ToChar' when targeting Dialect.
Argument 'format' is not supported for expression 'ToChar' when targeting Dialect.
Argument 'format' is not supported for expression 'ToChar' when targeting Dialect.
Argument 'format' is not supported for expression 'ToChar' when targeting Dialect.
Argument 'format' is not supported for expression 'ToChar' when targeting Dialect.
Argument 'format' is not supported for expression 'ToChar' when targeting Dialect.
Argument 'format' is not supported for expression 'ToChar' when targeting Dialect.
Argu

In [32]:
with open('pred_codegen_ua.sql','w') as f:
    for rec in ua_queries:
        line = f"{rec}\n"
        f.write(line)

## check parsability

In [35]:
import process_sql

In [43]:
fails = 0
for q, db in zip(pred_queries,single_table_queries) :
    try:
        database = f"spider_data/database/{db['db_id']}/{db['db_id']}.sqlite"
        schema = process_sql.get_schema(database)
        process_sql.get_sql(process_sql.Schema(schema), q)
    except:
        fails+=1

In [44]:
fails

3094

In [47]:
fails = 0
with open('pred_ua.sql') as f:
    qu = f.readlines()
    
for q, db in zip(qu,single_table_queries) :
    try:
        database = f"spider_data/database/{db['db_id']}/{db['db_id']}.sqlite"
        schema = process_sql.get_schema(database)
        process_sql.get_sql(process_sql.Schema(schema), q.strip())
    except:
        fails+=1

In [48]:
fails

1963

In [45]:
fails = 0
for q, db in zip(pred_codegen_queries,single_table_queries) :
    try:
        database = f"spider_data/database/{db['db_id']}/{db['db_id']}.sqlite"
        schema = process_sql.get_schema(database)
        process_sql.get_sql(process_sql.Schema(schema), q)
    except:
        fails+=1

In [46]:
fails

2853

In [49]:
fails = 0
with open('pred_codegen_ua.sql') as f:
    qu = f.readlines()
    
for q, db in zip(qu,single_table_queries) :
    try:
        database = f"spider_data/database/{db['db_id']}/{db['db_id']}.sqlite"
        schema = process_sql.get_schema(database)
        process_sql.get_sql(process_sql.Schema(schema), q.strip())
    except:
        fails+=1

In [50]:
fails

1832

## executability

In [51]:
with open('spider_gold.sql') as f:
    gold_sql = f.readlines()

In [59]:
import sqlite3
execution_results = []
for g,p,pc in zip(gold_sql, pred_queries, pred_codegen_queries):
    results = {}
    g = g.strip().split('\t')
    p = str(p).strip()
    pc = str(pc).strip()

    db = f"spider_data/database/{g[1]}/{g[1]}.sqlite"
    con = sqlite3.connect(db)

    try:
        df = pd.read_sql_query(g[0],con)
        results['gold_data'] = df
    except:
        results['gold_data'] = None

    try:
        df = pd.read_sql_query(p,con)
        results['pred_data'] = df
    except:
        results['pred_data'] = None

    try:
        df = pd.read_sql_query(pc,con)
        results['pred_codegen_data'] = df
    except:
        results['pred_codegen_data'] = None

    con.close()
    results['g'] = g
    results['p'] = p
    results['pc'] = pc

    execution_results.append(results)

In [60]:
gfails, pfails, pcfails = 0,0,0

for res in execution_results:
    if res['gold_data'] is None:
        gfails+=1

    if res['pred_data'] is None:
        pfails+=1

    if res['pred_codegen_data'] is None:
        pcfails+=1

In [61]:
gfails, pfails, pcfails

(2, 1290, 1156)

## execution_accuracy

In [ ]:
fails = 0
total_compare = 0
fails_list = []

for i, rec in enumerate(execution_results):
    if rec['gold_data'] is not None and rec['pred_data'] is not None:
        total_comapre+=1

        if rec['gold_data'].shape!=rec['pred_data'].shape:
            fails+=1
            fails_list.append[i]
        else:
            for col1 in rec['gold_data'].columns:
                match=False
                for col2 in rec['pred_data'].columns:
                    if (rec['gold_data'][col1].astype('str').sort_values().values==rec['pred_data'][col2].astype('str').sort_values().values).all():
                        match=True
                        break
                if not match:
                    break
            
            if not match:
                fails+=1
                fails_list.append(i)